In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_7/models/B5R2b5_AN_1FC_fold7_3.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image2/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold7_3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1058, 22)
(1058, 22)
Normal:  (598, 22)
Abnormal:  (460, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,414,414,49,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,271.0,118.0,530.0,603.0,0.275638,0.131439,0.538745,0.670707
1,1,417,417,49,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,178.0,111.0,680.0,450.0,0.177931,0.123359,0.678119,0.501010
2,2,421,421,49,P1,P1,Abnormal,AB05,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF1,https://irisprodseatraining.blob.core.windows....,109.0,109.0,699.0,594.0,0.107466,0.121338,0.685053,0.660606
3,3,422,422,49,P1,P1,Abnormal,AB06,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF2,https://irisprodseatraining.blob.core.windows....,351.0,61.0,595.0,458.0,0.345968,0.068813,0.586762,0.509091
4,4,455,455,21,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,93.0,96.0,617.0,555.0,0.095064,0.107041,0.628885,0.617530


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1058 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.15525924, -0.17941773,  0.1774025 , ...,  0.32674763,
         0.17614375, -0.14238034],
       [-0.16339284,  0.09165455, -0.12919679, ..., -0.16481623,
        -0.04375339, -0.08398885],
       [-0.14664246, -0.06563856, -0.12192266, ...,  0.20871505,
         0.1322358 , -0.15324178],
       ...,
       [ 0.68779033,  0.93208128, -0.16878083, ...,  0.08411677,
        -0.0690282 ,  0.52088296],
       [ 0.04476101,  0.76550645, -0.18256035, ..., -0.06513412,
        -0.01705365,  0.25438526],
       [-0.1189504 ,  0.55062354, -0.17048928, ..., -0.1689382 ,
        -0.05550657, -0.03460712]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.155259,-0.179418,0.177402,0.470569,-0.189734,-0.221299,0.162996,-0.189721,-0.220627,0.041606,...,-0.164935,0.221224,-0.219147,-0.169777,-0.200259,0.031554,-0.025925,0.326748,0.176144,-0.142380
1,-0.163393,0.091655,-0.129197,-0.129003,-0.061019,-0.217985,0.006068,-0.028274,-0.065155,-0.203133,...,-0.079798,-0.162656,-0.020704,0.037092,0.042246,-0.064836,-0.124081,-0.164816,-0.043753,-0.083989
2,-0.146642,-0.065639,-0.121923,0.078993,-0.212230,-0.181396,0.099635,-0.140027,-0.050937,-0.202273,...,-0.052941,0.360586,-0.147019,-0.166990,-0.197151,0.034495,0.430760,0.208715,0.132236,-0.153242
3,-0.010863,-0.068397,-0.076694,0.086390,-0.085575,-0.090764,0.286853,-0.100185,0.055757,-0.119907,...,-0.080531,-0.005231,0.074322,-0.014654,-0.206130,-0.108709,0.387218,-0.001178,0.092962,-0.175927
4,-0.053581,-0.070526,0.052108,0.135258,-0.090246,-0.099231,0.174165,-0.054822,-0.117625,-0.183266,...,0.002010,-0.057460,-0.130480,-0.020887,-0.061941,-0.195151,-0.038029,-0.118949,-0.055010,0.028432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,0.044761,0.765506,-0.182560,-0.153772,0.213677,0.567514,-0.167241,0.206704,0.495684,-0.176368,...,0.113299,-0.208388,0.175752,0.434782,0.534003,-0.197093,-0.144643,-0.065134,-0.017054,0.254385
1054,-0.141355,0.053651,-0.197149,-0.087201,0.336112,0.261595,-0.173413,-0.042208,0.143733,-0.172221,...,0.124821,-0.135495,0.097971,-0.135808,0.136451,-0.169053,-0.086879,-0.140537,-0.182326,0.114861
1055,0.687790,0.932081,-0.168781,0.041376,0.795956,1.141080,-0.138899,1.069099,0.790001,-0.149760,...,0.780366,-0.184156,0.865603,0.488194,1.212970,-0.170099,0.015881,0.084117,-0.069028,0.520883
1056,0.044761,0.765506,-0.182560,-0.153772,0.213677,0.567514,-0.167241,0.206704,0.495684,-0.176368,...,0.113299,-0.208388,0.175752,0.434782,0.534003,-0.197093,-0.144643,-0.065134,-0.017054,0.254385


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.155259,-0.179418,0.177402,0.470569,-0.189734,-0.221299,0.162996,-0.189721,-0.220627,0.041606,...,-0.200259,0.031554,-0.025925,0.326748,0.176144,-0.142380,Abnormal,P1,FP-A,AB02
1,-0.163393,0.091655,-0.129197,-0.129003,-0.061019,-0.217985,0.006068,-0.028274,-0.065155,-0.203133,...,0.042246,-0.064836,-0.124081,-0.164816,-0.043753,-0.083989,Abnormal,P1,FP-A,AB04
2,-0.146642,-0.065639,-0.121923,0.078993,-0.212230,-0.181396,0.099635,-0.140027,-0.050937,-0.202273,...,-0.197151,0.034495,0.430760,0.208715,0.132236,-0.153242,Abnormal,P1,FP-A,AB05
3,-0.010863,-0.068397,-0.076694,0.086390,-0.085575,-0.090764,0.286853,-0.100185,0.055757,-0.119907,...,-0.206130,-0.108709,0.387218,-0.001178,0.092962,-0.175927,Abnormal,P1,FP-A,AB06
4,-0.053581,-0.070526,0.052108,0.135258,-0.090246,-0.099231,0.174165,-0.054822,-0.117625,-0.183266,...,-0.061941,-0.195151,-0.038029,-0.118949,-0.055010,0.028432,Abnormal,P1,FP-A,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,0.044761,0.765506,-0.182560,-0.153772,0.213677,0.567514,-0.167241,0.206704,0.495684,-0.176368,...,0.534003,-0.197093,-0.144643,-0.065134,-0.017054,0.254385,Normal,P8,FP-E,Normal
1054,-0.141355,0.053651,-0.197149,-0.087201,0.336112,0.261595,-0.173413,-0.042208,0.143733,-0.172221,...,0.136451,-0.169053,-0.086879,-0.140537,-0.182326,0.114861,Normal,P8,FP-E,Normal
1055,0.687790,0.932081,-0.168781,0.041376,0.795956,1.141080,-0.138899,1.069099,0.790001,-0.149760,...,1.212970,-0.170099,0.015881,0.084117,-0.069028,0.520883,Normal,P8,FP-E,Normal
1056,0.044761,0.765506,-0.182560,-0.153772,0.213677,0.567514,-0.167241,0.206704,0.495684,-0.176368,...,0.534003,-0.197093,-0.144643,-0.065134,-0.017054,0.254385,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold7_3.csv')